In [6]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from conct_db import enginex
from datetime import date

**LEER LOS DATOS DE ORIGIEN //CARTERA DE REGISTROS//**

In [7]:
file = pd.read_excel('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsb', skipfooter=3, sheet_name='Export')
file.to_csv('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsx', encoding='UTF-8')
data1 = pd.read_csv('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.csv', encoding='UTF-8')
df = pd.DataFrame(data1)
# df.loc[[2134]]
# df.sample(3)
df.head()

,Código Distribuidor,Distribuidor,POC,Clientes,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles
0,70425,FANK Y SCHERF SRL.-,7.042500e+12,1.0,0.0,NaN,0.0,NaN,NaN,NO,NaN
1,70425,FANK Y SCHERF SRL.-,7.042500e+12,1.0,0.0,NaN,0.0,NaN,NaN,NO,NaN
2,70425,FANK Y SCHERF SRL.-,7.042500e+12,1.0,0.0,NaN,0.0,NaN,NaN,NO,NaN
3,70425,FANK Y SCHERF SRL.-,7.042500e+12,1.0,1.0,16/6/2022,1.0,1.0,16/6/2022,NO,2037.0
4,70425,FANK Y SCHERF SRL.-,7.042500e+12,1.0,0.0,NaN,0.0,NaN,NaN,NO,NaN


**TRANSFORMAR LOS DATOS PARA PODER CARGARLOS A LA BASE**

In [8]:
df['POC'] = df['POC'].astype(str)
# df['Fecha adhesion'] = df['Fecha adhesion'].convert_dtypes(int)
df['Fecha adhesion'] = pd.to_datetime(df['Fecha adhesion'],  format='%d/%m/%Y', errors='coerce')
df['Fecha registro'] = pd.to_datetime(df['Fecha registro'], format='%d/%m/%Y', errors='coerce', )
# df['Fecha adhesion'] = df['Fecha adhesion'].dt.strftime('%m/%d/%Y')

In [9]:
# df['COD_CLIENTE'] = df.POC.str[-5:]
df['COD_CLIENTE'] = df.POC.str.extract('(0{2,}\d+)')
df.rename({'Código Distribuidor':'COD_DISTRI'}, axis=1, inplace=True)
# df['COD_DISTRI'] = df.POC.str.extract('(\d{6,6}(?:\.\d+)?)')
df[['COD_CLIENTE','Registrado', 'Adherible', 'Adherido']] = df[['COD_CLIENTE','Registrado', 'Adherible', 'Adherido']].astype('Int64')
try:
    df.drop(columns=['POC', 'Distribuidor', 'Clientes'], inplace=True)
except:
    pass
df = df[['COD_DISTRI', 'COD_CLIENTE', 'Registrado', 'Fecha registro', 'Adherible', 'Adherido', 'Fecha adhesion', 'CanjeL3M', 'Puntos Disponibles']]
df.sample(3)

,COD_DISTRI,COD_CLIENTE,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles
6617,369846,7047,1,2022-05-20,1,1,2022-05-20,NO,1715.0
865,328660,1078,1,2021-12-07,1,1,2021-12-07,SI,2756.0
4241,369117,14071,1,2022-01-14,1,1,2022-01-14,NO,1915.0


In [10]:

try:
    df.to_sql('BEES_Cartera_Registros', con=enginex, if_exists='replace', index=False)
    df.to_excel('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsx')
    print('Carga Exitosa')
except Exception as ex:
    print(ex)
    

Carga Exitosa
